파이썬 머신러닝 완벽 가이드 `ch3 1~5` `pg.147~174`를 필사

**결정 클래스 값 종류의 유형에 따른 분류**

- 이진 분류 : 2개의 결괏값(긍정/부정)만 가짐

- 멀티 분류 : 여러 개의 결정 클래스 값을 가짐

​

**분류의 성능 평가 지표 (이진, 멀티 분류에 모두 적용되나 이진 분류에서 더욱 중요)**

- 정확도(Accuracy)

- 오차행렬(Confusion Matrix)

- 정밀도(Precision)

- 재현율(Recall)

- F1 스코어

- ROC AUC

# 01. 정확도(Accuracy)

## 1) 정확도

- 실제 데이터와 예측 데이터가 같은 정도를 판단하는 지표

- 직관적으로 모델 예측 성능 나타냄

- 이진 분류 : 데이터 구성에 따라 머신러닝 모델 성능 왜곡 가능, ∴ 정확도만 이용하여 성능 평가하지 x

- 불균형한 레이블 값 분포에서 머신러닝 모델 성능 판단할 경우 부적합 (ex. 데이터 100개 中 0 레이블 데이터*90, 1 레이블 데이터 *10 인 경우 : 무조건 0으로 예측 결과 반환하는 모형도 정확도 90%)

\
## 2) 불균형한 레이블 값 분포의 예 : MNIST 데이터 세트

- MNIST 데이터 세트 : 0~9 숫자 이미지의 픽셀 정보 有. 이를 기반으로 숫자 digit 예측

- 사이킷런 load_digists() :  MNIST 데이터 세트 제공, 멀티 레이블 분류

- 7만 True, 나머지는 모두 False로 변환해 이진 분류 문제로 설정 (= 전체의 10%만 True)

- 모든 것을 0으로만 예측해도 정확도 90% → 불균형한 레이블 데이터 세트에서 정확도를 성능 수치로 사용해서는 안됨

In [1]:
import sklearn

print(sklearn.__version__)

1.0.2


In [2]:
import numpy as np
from sklearn.base import BaseEstimator

class MyDummyClassifier(BaseEstimator):
    # fit( ) 메소드는 아무것도 학습하지 않음. 
    def fit(self, X , y=None):
        pass
    
    # predict( ) 메소드는 단순히 Sex feature가 1 이면 0 , 그렇지 않으면 1 로 예측함. 
    def predict(self, X):
        pred = np.zeros( ( X.shape[0], 1 ))
        for i in range (X.shape[0]) :
            if X['Sex'].iloc[i] == 1:
                pred[i] = 0
            else :
                pred[i] = 1
        
        return pred

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 원본 데이터를 재로딩, 데이터 가공, 학습 데이터/테스트 데이터 분할
titanic_df = pd.read_csv("titanic_train.csv")
y_titanic_df = titanic_df['Survived']
X_titanic_df = titanic_df.drop('Survived', axis=1)
X_titanic_df = transform_features(X_titanic_df)
X_train, X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic_df,
                                                    test_size=0.2, random_state=0)

# 위에서 생성한 Dummy Classifier를 이용해 학습/예측/평가 수행
myclf = MyDummyClassifier()
myclf.fit(X_train, y_train)
mypredictions = myclf.predict(X_test)
print('Dummy Classifier의 정확도는: {0:.4f}'.format(accuracy_score(y_test, mypredictions)))

Dummy Classifier의 정확도는: 0.7877


In [5]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

class MyFakeClassifier(BaseEstimator):
  def fit(self, X, y):
    pass
  
  # 입력값으로 들어오는 X 데이터 세트의 크기만큼 모두 0값으로 만들어서 반환
  def predict(self, X):
    return np.zeros((len(X), 1), dtype=bool)
  

# 사이킷런의 내장 데이터 세트인 load_digits()를 이용해 MNIST 데이터 로딩
digits = load_digits()

#digits 번호가 7번이면 True이고 이를 astype(int)로 1로 변환, 7번이 아니면 False이고 0으로 변환
y = (digits.target == 7).astype(int)
X_train, X_test, y_train, y_test = train_test_split(digits.data, y, random_state=11)

In [6]:
# 불균형한 레이블 데이터 분포도 확인
print("레이블 테스트 세트 크기:", y_test.shape)
print('테스트 세트 레이블 0과 1의 분포도')
print(pd.Series(y_test).value_counts())

# Dummy Classifier로 학습/예측/정확도 평가
fakeclf = MyFakeClassifier()
fakeclf.fit(X_train, y_train)
fakepred = fakeclf.predict(X_test)
print("모든 예측을 0으로 하여도 정확도는 :{:.3f}".format(accuracy_score(y_test, fakepred)))

레이블 테스트 세트 크기: (450,)
테스트 세트 레이블 0과 1의 분포도
0    405
1     45
dtype: int64
모든 예측을 0으로 하여도 정확도는 :0.900


# 02. 오차 행렬

## 1) 오차 행렬

① 이진 분류의 '예측 오류의 정도 + 예측 오류의 유형' 함께 나타내는 지표

② 4분면 행렬에서 실제 레이블 클래스 값, 예측 레이블 클래스 값의 유형 매핑

- 1사분면 : False positive(실제는 negative0, 예측은 positive1) *type1 error

- 2사분면 : True negative(실제도 negative0, 예측도 negative0)

- 3사분면 : Flase negative(실제는 positive1, 예측은 negative0) *type2 error

- 4사분면 : True positive(실제도 positive1, 예측도 positive1)

## 2) 사이킷런 - confusion_matrix()

① 오차행렬 구하는 api

② ndarray 형태로 결과물 출력

③ mnist 예제의 예측성능지표 오차행렬로 표시
- True negative 405건 : 전체 450건 데이터 중 7이 아닌(negative) 데이터 405건을 negative로 예측한 경우

- False negative 45건 : 전체 450건 데이터 중 7인(positive) 데이터 45건을 negative로 예측한 경우

- True positive, False positive 0건 : positive로 예측한 건수 없으므로 0건

## 3) 불균형한 레이블 클래스를 가지는 이진 분류 모델의 경우

① 중점적으로 찾아야하는 positive(1) 개수가 매우 적음

② 머신러닝 알고리즘은 데이터에 기반 → positive보다 negative로 정확도 상승

③ negative로 예측하는 경향 높아짐 → True positive 매우 감소, True negative 매우 증가

④ positive에 대한 예측 정확도 판단 x인 상태에서 negative에 대한 예측 정확도 만으로도 분류 정확도 높게 측정됨

⑤ ∴ 불균형 데이터 세트에서 선호되는 평가 지표 : 정밀도(precision), 재현율(recall)

03. 정밀도와 재현율

1) 정밀도(Precision)

① TP / (FP + TP) : positive 예측 中 실제로 positive인 경우

② 양성예측도

③ 실제 negative인 데이터를 positive로 예측했을 때(False positive) 업무상 큰 영향 발생하는 경우 중요 (ex. 스팸메일 : 스팸이 아닌 경우를 스팸이 맞다고 판단해 휴지통에 넣어버리는 경우 문제 발생)

④ FP를 낮추고 TP를 높이는 데 초점

⑤ 사이킷런 api : precision_score()

2) 재현율(Recall)

① TP / (FN + TP) : 실제 positive인 경우 中 예측도 positive인 경우

② 민감도(Sensitivity), TPR(True Positive Rate)

③ 실제 positive인 데이터를 negative로 예측했을 때(False negative) 업무상 큰 영향 발생하는 경우 중요 (ex. 암 판단 모델, 금융 사기 적발 모델)

④ FN을 낮추고 TP를 높이는 데 초점

⑤ 사이킷런 api : recall_score()

3) 정밀도/재현율 트레이드오프

① 정밀도/재현율 트레이드 오프 : 정밀도와 재현율 중 어느 한 쪽 강제로 높이면 다른 쪽은 감소(∵상호보완관계)

② 사이킷런 predict_proba() 

- 개별 데이터별로 예측 '확률' 반환 (cf. predict() : 예측결과 클래스값 반환)

- 학습 완료된 사이킷런 classifier 객체에서 호출 가능

- 테스트 피처 데이터 세트를 파라미터로 입력 / 테스트 피처 레코드의 개별 클래스 예측 확률 반환

- 반환 형태 : ndarray. 1열 - 클래스값 0에 대한 예측 확률 / 2열 - 클래스값 1에 대한 예측 확률

- predict() : predict_proba()가 반환한 ndarray에서 확률 큰 열의 위치 받아 최종 예측 클래스 결정

③ 사이킷런 Binarizer

- fit_transform() : ndarray 입력시 지정된 threshold보다 작으면 0, 크면 1 반환

④ 임곗값 ↓, 재현율↑, 정밀도↓ (∵임곗값 : positive 예측값 결정 기준, 낮아질 수록 True 값 多)

⑤ 사이킷런 precision_recall_curve()

- 입력 파라미터 : y_true(실제 클래스값 배열), probas_pred(positive 칼럼의 예측확률 배열)

- 반환값 : 정밀도, 재현율, 임곗값 0.11~0.95(임곗값 일부분만 추출 가능)

- 정밀도, 재현율 곡선 시각화 가능

## 4) 정밀도와 재현율의 맹점

① 정밀도=100% 되는 방법

- 확실한 기준이 되는 경우만 positive로 예측, 나머지는 모두 negative로 예측

- 예시 : 전체 환자 1000명 중 확실한 positive 징후 가진 환자 1명만 positive로 예측, 나머지 999명은 모두 negative로 예측

            → FP=0, TP=1. ∴ TP / (FP+TP) = 1/1 = 1(100%)

② 재현율=100% 되는 방법

- 모든 경우를 positive로 예측

- 예시 : 전체 환자 1000명을 모두 positive로 예측 → FN=0. ∴ TP / (FN+TP) = TP/(0+TP) = 1(100%)

# 04. F1 score
 
## 1) F1 score

① 정밀도&재현율 결합한 지표

② 정밀도, 재현율 둘중 어느 곳으로도 치우치지 않을 때 높은 값 가짐

③ 공식
$$ 2 \times \frac{precision \times recall}{precision + recall} $$

④ 사이킷런 API : f1_score()

# 05. ROC curve, AUC

## 1) ROC curve

① 머신러닝 이진분류모델의 예측 성능 판단하는 중요 평가 지표

② FPR이 변할 때 TPR이 변하는 양상 보여줌

③ FPR

- False Positive Rate

- X축

- FP / (FP+TN) = 1-TNR = 1-특이성

④ TPR

- True Positve Rate

- Y축

- TP / (FN+TP) : 재현율.

- positive가 정확히 예측되어야 하는 수준 (질병 있는 사람은 질병 있다고 양성 판정)

⑤ TNR

- True Negative Rate

- TN / (FP+TN) : 특이성. 

- negative가 정확히 예측되어야 하는 수준 (질병 없는 사람은 질병 없다고 음성 판정)

⑥ 분류 결정 임곗값을 변경해 FPR을 0부터 1까지 변경하며 TPR의 변화값 구함

- 분류결정임곗값 : positive 예측값 결정하는 확률의 기준

- FPR=0 : 분류 결정 임곗값=1 (∵positive로 예측하는 것 없음)

- FPR=1 : 분류 결정 임곗값=0 (∵모든 것을 positive로 예측)

⑦ 사이킷런 API : roc_curve()

- 반환값 : FPR, TPR, 임곗값

- 입력 파라미터 : y_true(실제 클래스값 array), y_score(predict_proba() 반환값 array의 positive 칼럼 예측 확률)

## 2) AUC(Area Under Curve)

① ROC 곡선 밑의 면적. 1에 가까울 수록 좋은 수치

② 분류의 성능 지표로 이용 (cf. ROC curve는 fpr과 tpr의 변화 양상을 보기 위해 사용)

③ 보통의 분류는 0.5(랜덤 수준의 이진분류 auc값) 이상의 값을 가짐

④ 사이킷런 API : roc_auc_score()